In [1]:
import torch
import pykeops

In [2]:
torch.cuda.current_device()

0

In [8]:
device = 'cuda:0'

torch.manual_seed(0)
ntr = 1000
ntst = 100
d = 3

xtr = torch.randn(ntr,d).to(device)
xtst = torch.randn(ntst,d).to(device)
btr = torch.randn(ntr).to(device)
btst = torch.randn(ntst).to(device)

In [9]:
from opts.pcg import PCG

In [10]:
kernel_params = {"type": "rbf", "sigma": 1}
gauss_nys_params = {"type": "nystrom", "r": 100, "rho": 1}
lambd = 1
task = 'test_mse'
a0 = torch.zeros(ntr).to(device)
pcg_iters = 10

In [11]:
#Test Nystrom PCG with Gaussian test matrix
nystrom_pcg = PCG(gauss_nys_params)
a = nystrom_pcg.run(xtr,btr,xtst,btst,kernel_params,lambd, task,
                    a0,pcg_iters,device,None)

[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
PCG has converged with residual:tensor(1.3815e-07, device='cuda:0')


In [12]:
#Test Nystrom PCG with greedy column selection
greedy_nys_params = {"type": "pivoted_cholesky", "r": 100, "rho": 1}
greedy_nystrom_pcg = PCG(greedy_nys_params)
a = greedy_nystrom_pcg.run(xtr,btr,xtst,btst,kernel_params,lambd, task,
                           torch.zeros(1000).to(device),pcg_iters,device,None)

PCG has converged with residual:tensor(8.8497e-07, device='cuda:0')
